In [1]:
import cv2
import os
import numpy as np

In [2]:
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    if (len(faces) == 0) :
        return None, None
    x, y ,w, h = faces[0]
    return gray[y:y+w, x:x+h], faces[0]

In [3]:
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
 
    faces = []
    labels = []
    
    for dir_name in dirs:
            
        if not dir_name.startswith("s"):
            continue
        
        label = int(dir_name.replace("s", ""))

        subject_dir_path = data_folder_path + "/" + dir_name
        subject_images_names = os.listdir(subject_dir_path)
        
        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue
            
            image_path = subject_dir_path + "/" + image_name

            image = cv2.imread(image_path)
            
            # cv2.imshow("Training on image...", image)
            # cv2.waitKey(100)
            
            face = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            if face is not None:
                faces.append(face)
                labels.append(label)
    
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [4]:
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

print("Total faces:", len(faces))
print("Total labels:", len(set(labels)))

Preparing data...
Data prepared
Total faces: 557
Total labels: 2


In [34]:
face_recognizer = cv2.face.EigenFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

In [13]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

In [14]:
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [38]:
dirs = os.listdir("data-tes")
for image_name in dirs:
    test_img = cv2.imread("data-tes/" + image_name)
    img = test_img.copy()
    face, rect = detect_face(img)
    face = cv2.resize(face,(200,200))

    subjects = ["", "female", "male"]

    label, confidence = face_recognizer.predict(face)
    label_text = subjects[label]

    draw_rectangle(img, rect)
    draw_text(img, label_text, rect[0], rect[1]-5)

    cv2.imshow('hasil', img)
    cv2.imwrite("hasil/" + image_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()